## Installation and deployment of 5G Core (5GC) of Open Air Interface (OAI)
## Objectives
- Install and configure the 5GC of OAI in a virtual machine Ubuntu 20.04 LTS (Focal Fossa)
- Deploy 5GC Core
- Inspec 5G Core Docker containers by means of command line instructions 
- Install and configure Portainer tool
- Inspec 5G Core Docker containers by means of Portainer browser

At the end of a set of three hands-on (this is the first one), the goal is to run the simulated OAI scenario, which consists of the 5G core (5GC), gNB in CU/DU split mode, and user equipment (UE), according to the image below: 

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy.jpg)

Now, we are going to install, configure, deploy and interact to 5GC.


## General Instructions

In this task you will install and configure the core in its basic architecture according to the image below:

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_01.png)

**Minimum hardware requirements for OAI UE, according to doc/NR_SA_Tutorial_COTS_UE.md · develop · oai / openairinterface5G · GitLab (eurecom.fr):**
- Ubuntu 20.04.5 LTS;
- CPU: 4 cores x86_64;
- 8 GB RAM.

**Minimum hardware requirements for OAI 5GC and OAI gNB, according to doc/NR_SA_Tutorial_COTS_UE.md · develop · oai / openairinterface5G · GitLab (eurecom.fr):**
- Ubuntu 20.04.5 LTS;
- CPU: 8 cores x86_64 3.5 GHz;
- 32 GB RAM.

**However, if you do not have such hardware specifications, you may use the one below for both devices (tested for us, you may face some lags, but it works):**
- Ubuntu 20.04.5 LTS;
- CPU: 4 cores x86_64;
- 4 GB RAM.

## 5GC Installation

In a Linux Terminal, follow the steps below:

**Step 0:** Setup your VM or Baremetal Machine, after install Useful Software:

sudo su

apt update -y

apt upgrade -y

apt install -y python3-pip git curl cmake tree golang vim net-tools build-essential nfs-common openssh-server

**Step 1:** Install dependencies

sudo apt-get install ca-certificates 

sudo apt-get install curl 

sudo apt-get install gnupg 

sudo apt-get install lsb-release 

**Step 2:** Enable packet forwarding

sudo sysctl net.ipv4.conf.all.forwarding=1

sudo iptables -P FORWARD ACCEPT

**Step 3:** Install docker and docker-compose

sudo rm /etc/apt/sources.list.d/docker.list*

sudo mkdir -p /etc/apt/keyrings 

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu \\$(. /etc/os-release; echo "\\$UBUNTU_CODENAME") stable"

sudo chmod a+r /etc/apt/keyrings/docker.gpg

sudo apt-get update

sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin

```python 
sudo curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose
```

sudo chmod +x /usr/local/bin/docker-compose

**Step 4:** Pull the images from Docker Hub

sudo docker pull oaisoftwarealliance/oai-amf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-nrf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-smf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udr:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udm:v1.5.0

sudo docker pull oaisoftwarealliance/oai-ausf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-spgwu-tiny:v1.5.0

sudo docker pull oaisoftwarealliance/trf-gen-cn5g:latest

**Step 5:** Tag Docker Images

sudo docker image tag oaisoftwarealliance/trf-gen-cn5g:latest trf-gen-cn5g:latest

sudo docker image tag oaisoftwarealliance/oai-amf:v1.5.0 oai-amf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-nrf:v1.5.0 oai-nrf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-smf:v1.5.0 oai-smf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udr:v1.5.0 oai-udr:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udm:v1.5.0 oai-udm:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-ausf:v1.5.0 oai-ausf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-spgwu-tiny:v1.5.0 oai-spgwu-tiny:v1.5.0

**Step 6:** Clone OpenAirInterface 5G Core repository

git clone https://gitlab.eurecom.fr/oai/cn5g/oai-cn5g-fed.git

cd oai-cn5g-fed

git checkout -f v1.5.0

cd ..




## 5GC Configuration

In a Linux Terminal, follow the steps below:

**Step 1:** Configuration

The configuration will be performed using docker-compose. First, run the command below to download and copy the OAI core docker-compose file. We consider that <OAI_PATH> (path that you install OAI) is **/home/dco1020** (the file will already be placed in the correct path <OAI_PATH>/oai-cn5g-fed/docker-compose/). Please, change <OAI_PATH> by **/home/dco1020** in the following commands. In case you install in a different folder, change commands properly.

git clone https://github.com/vicentesousa/DCO1020.git

cp ./DCO1020/CODES/H01_5GCore_UNI_III/docker-compose-basic-nrf.yaml  /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml

Then copy the database used for this deployment. This database contains important information such as data for gNB and UE authentication. The command below will download and place the file already in the correct path (PATH_TO/oai-cn5g-fed/docker-compose/database/).

cp /home/dco1020/DCO1020/CODES/H01_5GCore_UNI_III/oai_db.sql /home/dco1020/oai-cn5g-fed/docker-compose/database/oai_db.sql 

**Step 2:** Synchronizing the tutorials

cd /home/dco1020/oai-cn5g-fed/

sudo ./scripts/syncComponents.sh

sudo git submodule deinit --all --force

sudo git submodule init

sudo git submodule update

**Step 3:** Create the bridge 5GC

The command below will then create a subnet that will provide communication between the containers in the core.

sudo docker network create --driver=bridge --subnet=192.168.70.128/26 -o "com.docker.network.bridge.name"="demo-oai" demo-oai-public-net

## Deployment

For this task, open another terminal using crtl+Alt+T.

**Step 1:** Core deployment

cd /home/dco1020/oai-cn5g-fed/docker-compose

sudo python3 core-network.py --type start-basic --scenario 1

Expected result:

After running the above command, nine containers are expected to be in the "Up (healthy)" state as shown in figure below.

![5GC_deploy_02](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_02.png)

## Verificação dos Pods com linha de comando e com o Portainer

The 5GC components were installed as docker containers. A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another. A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings. Container images become containers at runtime and in the case of Docker containers – images become containers when they run on Docker Engine. Available for both Linux and Windows-based applications, containerized software will always run the same, regardless of the infrastructure. Containers isolate software from its environment and ensure that it works uniformly despite differences for instance between development and staging.

Containers and virtual machines have similar resource isolation and allocation benefits, but function differently because containers virtualize the operating system instead of hardware. Containers are more portable and efficient. Figure bellow illustrates the difference between VMs and containers.

![5GC_deploy_04](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_04.png)

**Step 1**: to see all 5GC docker containers by command line you could type the following command in a terminal:

docker ps -a

The results are as bellow:

![5GC_deploy_03](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_03.png)

You can see all virtual functions that comprise the 5GC.


**Step 2**: let's logs of a running docker container, for instance, let's see logs of the 5GC's AMF container. In a terminal , type:

docker logs -f 2de7bfa9afa4

You will see a screen similar to the figure bellow.

![5GC_deploy_05](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_05.png)


You can find the same results with the command:

docker logs -f oai-amf

**Step 3**: you can also see the hardware resources all containers are consuming. In a terminal , type:

docker container stats

You will see a screen similar to the figure bellow.

![5GC_deploy_06](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_06.png)

To return to terminal, type: crtl+c

**Step 4**: let's execute a command inside a running docker container, for instance, let's open a bash inside the 5GC's AMF container and see the processes in execution. See in figures above that **2de7bfa9afa4** is the container ID of the 5GC's AMF container. So, in a terminal , type (ps.: realize that you need to see the containerID of 5GC's AMF installed in your machine):

docker exec -it 2de7bfa9afa4 bash

You will see a screen similiar to the figure bellow.

![5GC_deploy_07](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_07.png)

As you have a bash shell inside the container, you can type bash commands. Let's see all processes running by this container. Type:

top

You will see a screen similiar to the figure bellow.

![5GC_deploy_08](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_08.png)

To return to terminal, type: crtl+c

**Step 5 (optional)**: To stop the core network you can use:

python3 core-network.py --type stop-basic --scenario 1


A good Docker Cheat Sheet can be found [here](https://docs.docker.com/get-started/docker_cheatsheet.pdf).


**Step 6 (optional)**: there is a plenty of graphical tool to manager, see and operate dockers. Let's install and use Portainer. To install it, first, create the volume that Portainer Server will use to store its database. Ia a terminal, type:

docker volume create portainer_data

**Step 7 (optional)**: Then, download and install the Portainer Server container:

docker volume create portainer_data

docker run -d -p 8000:8000 -p 9443:9443 --name portainer --restart=always -v /var/run/docker.sock:/var/run/docker.sock -v portainer_data:/data portainer/portainer-ce:latest

More details [in this link.](https://docs.portainer.io/start/install-ce/server/docker/linux)

**Step 8 (optional)**: Now that the installation is complete, you can log into your Portainer Server instance by opening a web browser and going to:

https://localhost:9443


**Step 9 (optional)**: You need to create a user. For this, you will see a screen similar to:

![5GC_port_8](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_08.png)

**Step 10 (optional)**: Now, you can see the initial screen and you could proceedd using the local environment. Please, click on **Get Started**.

![5GC_port_9](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_09.png)

**Step 11 (optional)**: then, you can see the screen with the local environment. Click on it.

![5GC_port_10](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_10.png)

**Step 12 (optional)**: Now, you could see the environment dashboard with all information about your Portainer. Let's see all Containers clicking on Containers menu as indicating below.

![5GC_port_11](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_11.png)

**Step 13 (optional)**: you can see all containers, their status and a menu of quick actions. 

![5GC_port_12](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_12.png)

**Step 14 (optional)**: To see the AMF container logs, please click on icone indicated in the figure below.

![5GC_port_13](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_13.png)

**Step 15 (optional)**: Finally, you can see AMF logs (note that you can filter and download it).

![5GC_port_14](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_14.png)

An interesting video about Portainer tool is: [Controle seus containers de maneira SIMPLES com o Portainer - Diolinux Labs](https://www.youtube.com/watch?v=q0LrLN4fylw),

# Deliverable 01:
See the log of all docker containers of 5GC and identify the following information:

- GUAMI (MCC, MNC, Region ID, AMF Set ID, AMF pointer):
- Slice Support: what are SST, SD supported? Explain those parameters. Indicate the 5QI of each slice.
- What is the N2 IP Address and port?
- What is the NRF IP Address and port?
- What is the AUSF IP Address and port?
- What is the AMF IP Address and port?
- What is the N4 IP Address and port?

Show the logs that you see all information.